In [62]:
import numpy as np
import pandas as pd
import openpyxl
import statsmodels.api as sm
import datetime
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

年度のデータフレーム準備

In [63]:
years = list(range(2009, 2021))
year_df = pd.DataFrame(years)
year_df.rename(columns={0: '年度'}, inplace=True)
year_df

,年度
0,2009
1,2010
2,2011
3,2012
4,2013
5,2014
6,2015
7,2016
8,2017
9,2018


人口関係のデータフレーム準備

In [64]:
Population_df = pd.read_excel('SSDSE-B-2023.xlsx', 
                         skiprows=1,
                         names=['年度', '地域コード', '都道府県', '総人口', '総人口（男）', '総人口（女）', '日本人人口', '日本人人口（男）', '日本人人口（女）', '15歳未満人口', '15歳未満人口（男）', '15歳未満人口（女）', '15～64歳人口', '15～64歳人口（男）', '15～64歳人口（女）', '65歳以上人口', '65歳以上人口（男）', '65歳以上人口（女）', '出生数', '出生数（男）', '出生数（女）', '合計特殊出生率', '死亡数', '死亡数（男）', '死亡数（女）', '転入者数（日本人移動者）', '転入者数（日本人移動者）（男）', '転入者数（日本人移動者）（女）', '転出者数（日本人移動者）', '転出者数（日本人移動者）（男）', '転出者数（日本人移動者）（女）', '婚姻件数', '離婚件数', '年平均気温', '最高気温（日最高気温の月平均の最高値）', '最低気温（日最低気温の月平均の最低値）', '降水日数（年間）', '降水量（年間）', '着工建築物数', '着工建築物床面積', '旅館営業施設数（ホテルを含む）', '旅館営業施設客室数（ホテルを含む）', '標準価格（平均価格）（住宅地）', '標準価格（平均価格）（商業地）', '幼稚園数', '幼稚園教員数', '幼稚園在園者数', '小学校数', '小学校教員数', '小学校児童数', '中学校数', '中学校教員数', '中学校生徒数', '中学校卒業者数', '中学校卒業者のうち進学者数', '高等学校数', '高等学校教員数', '高等学校生徒数', '高等学校卒業者数', '高等学校卒業者のうち進学者数', '短期大学数', '大学数', '短期大学教員数', '大学教員数', '短期大学学生数', '大学学生数', '短期大学卒業者数', '短期大学卒業者のうち進学者数', '大学卒業者数', '大学卒業者のうち進学者数', '専修学校数', '各種学校数', '専修学校生徒数', '各種学校生徒数', '新規求職申込件数（一般）', '月間有効求職者数（一般）', '月間有効求人数（一般）', '充足数（一般）', '就職件数（一般）', '一般旅券発行件数', '延べ宿泊者数', '外国人延べ宿泊者数', '着工新設住宅戸数', '着工新設持家数', '着工新設貸家数', '着工新設分譲住宅数', '着工新設住宅床面積', '着工新設持家床面積', '着工新設分譲住宅床面積', '着工新設貸家床面積', 'ごみ総排出量（総量）', '1人1日当たりの排出量', 'ごみのリサイクル率', '一般病院数', '一般診療所数', '歯科診療所数', '保育所等数', '保育所等定員数', '保育所等利用待機児童数', '保育所等在所児数', '保育所等保育士数', '消費支出（二人以上の世帯）', '食料費（二人以上の世帯）', '住居費（二人以上の世帯）', '光熱・水道費（二人以上の世帯）', '家具・家事用品費（二人以上の世帯）', '被服及び履物費（二人以上の世帯）', '保健医療費（二人以上の世帯）', '交通・通信費（二人以上の世帯）', '教育費（二人以上の世帯）', '教養娯楽費（二人以上の世帯）', 'その他の消費支出（二人以上の世帯）'])
Population_df = Population_df[Population_df['都道府県'] == '埼玉県']

columns_to_keep = ['年度', '総人口', '総人口（男）', '総人口（女）', '日本人人口', '日本人人口（男）', '日本人人口（女）', '15歳未満人口', '15歳未満人口（男）', '15歳未満人口（女）', '15～64歳人口', '15～64歳人口（男）', '15～64歳人口（女）', '65歳以上人口', '65歳以上人口（男）', '65歳以上人口（女）', '出生数', '出生数（男）', '出生数（女）', '合計特殊出生率', '死亡数', '死亡数（男）', '死亡数（女）']
Population_df = Population_df.drop(columns=[col for col in Population_df.columns if col not in columns_to_keep])
Population_df = Population_df.iloc[::-1]  # インデックスを逆順にする
Population_df.reset_index(drop=True, inplace=True)

Population_df

,年度,総人口,総人口（男）,総人口（女）,日本人人口,日本人人口（男）,日本人人口（女）,15歳未満人口,15歳未満人口（男）,15歳未満人口（女）,...,65歳以上人口,65歳以上人口（男）,65歳以上人口（女）,出生数,出生数（男）,出生数（女）,合計特殊出生率,死亡数,死亡数（男）,死亡数（女）
0,2009,7161000,3597000,3565000,7074000,3557000,3516000,969000,496000,473000,...,1427000,657000,771000,59725,30754,28971,1.28,52374,29123,23251
1,2010,7194556,3608711,3585845,7054944,3540768,3514176,953668,488332,465336,...,1464860,668635,796225,59437,30805,28632,1.32,55487,30709,24778
2,2011,7209000,3612000,3597000,7119000,3573000,3546000,949000,486000,463000,...,1506000,687000,819000,58059,29918,28141,1.28,57670,31525,26145
3,2012,7216000,3612000,3604000,7130000,3574000,3555000,940000,482000,458000,...,1585000,723000,862000,56943,29155,27788,1.29,59137,32440,26697
4,2013,7228000,3614000,3614000,7140000,3576000,3564000,934000,479000,455000,...,1661000,757000,904000,57470,29418,28052,1.33,60264,32859,27405
5,2014,7247000,3621000,3626000,7151000,3579000,3572000,929000,476000,452000,...,1737000,792000,945000,55765,28703,27062,1.31,61269,33287,27982
6,2015,7266534,3628418,3638116,7111168,3553767,3557401,910805,466718,444087,...,1788735,812816,975919,56077,28740,27337,1.39,62565,34312,28253
7,2016,7288000,3637000,3652000,7168000,3580000,3587000,907000,465000,442000,...,1857000,843000,1014000,54447,28111,26336,1.37,63466,34667,28799
8,2017,7307000,3643000,3664000,7171000,3580000,3592000,899000,461000,438000,...,1900000,862000,1039000,53069,27008,26061,1.36,65764,35789,29975
9,2018,7325000,3650000,3675000,7171000,3575000,3595000,891000,456000,434000,...,1934000,875000,1059000,51241,26296,24945,1.34,67726,37154,30572


就業率のデータフレーム準備

In [90]:
ages = ['15歳以上', '15～64歳', '15～24歳', '15～19歳', '20～24歳', '25～34歳', '25～29歳', '30～34歳', '35～44歳', '35～39歳', '40～44歳', '45～54歳', '45～49歳', '50～54歳', '55～64歳', '55～59歳', '60～64歳', '65歳以上']
def multiply_specific_column(df, column_name, multiplier):
    df[column_name] = df[column_name].apply(lambda x: x * multiplier)

In [91]:
rows_to_read_1 = 23
Employment_woman_df = pd.read_excel('FEH_00200531_230720133248.xlsx', 
                         skiprows=14, nrows=rows_to_read_1, 
                         names=['時間軸コード', '年度', '', '15歳以上', '15～64歳', '15～24歳', '15～19歳', '20～24歳', '25～34歳', '25～29歳', '30～34歳', '35～44歳', '35～39歳', '40～44歳', '45～54歳', '45～49歳', '50～54歳', '55～64歳', '55～59歳', '60～64歳', '65歳以上'])

Employment_woman_df.reset_index(drop=True, inplace=True)
Employment_woman_df['年度'] = Employment_woman_df['年度'].str.replace('年', '').astype('int64')
multiply_specific_column(Employment_woman_df, ages, 10000)

Employment_woman_df

c:\Users\human\DataScience_FEA\venv\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,時間軸コード,年度,,15歳以上,15～64歳,15～24歳,15～19歳,20～24歳,25～34歳,25～29歳,...,35～44歳,35～39歳,40～44歳,45～54歳,45～49歳,50～54歳,55～64歳,55～59歳,60～64歳,65歳以上
0,2000000000,2000,NaN,6610000,**********************************************...,880000,150000,730000,1620000,950000,...,1140000,560000,580000,1590000,720000,870000,1030000,660000,370000,350000
1,2001000000,2001,NaN,6710000,**********************************************...,840000,160000,680000,1700000,970000,...,1190000,600000,590000,1590000,690000,900000,1020000,640000,380000,360000
2,2002000000,2002,NaN,6770000,**********************************************...,860000,140000,710000,1690000,920000,...,1240000,630000,610000,1540000,670000,870000,1060000,650000,420000,380000
3,2003000000,2003,NaN,6800000,**********************************************...,810000,150000,660000,1690000,910000,...,1280000,650000,620000,1490000,650000,840000,1130000,700000,430000,400000
4,2004000000,2004,NaN,6890000,**********************************************...,790000,140000,650000,1740000,910000,...,1340000,680000,670000,1410000,650000,760000,1200000,730000,470000,410000
5,2005000000,2005,NaN,6920000,**********************************************...,780000,160000,630000,1690000,880000,...,1420000,710000,710000,1380000,660000,720000,1220000,750000,470000,420000
6,2006000000,2006,NaN,6990000,**********************************************...,770000,160000,610000,1680000,870000,...,1490000,760000,730000,1360000,680000,680000,1240000,790000,450000,450000
7,2007000000,2007,NaN,7140000,**********************************************...,780000,140000,640000,1640000,830000,...,1590000,810000,780000,1400000,720000,680000,1260000,780000,480000,480000
8,2008000000,2008,NaN,7200000,**********************************************...,780000,140000,640000,1630000,830000,...,1650000,850000,790000,1390000,730000,670000,1250000,730000,520000,500000
9,2009000000,2009,NaN,7180000,6666666666666666666666666666666666666666666666...,750000,160000,590000,1600000,820000,...,1670000,860000,820000,1400000,730000,670000,1220000,690000,530000,530000


In [92]:
rows_to_read_2 = 23
Employment_man_df = pd.read_excel('FEH_00200531_230720133159.xlsx', 
                         skiprows=14, nrows=rows_to_read_2, 
                         names=['時間軸コード', '年度', '', '15歳以上', '15～64歳', '15～24歳', '15～19歳', '20～24歳', '25～34歳', '25～29歳', '30～34歳', '35～44歳', '35～39歳', '40～44歳', '45～54歳', '45～49歳', '50～54歳', '55～64歳', '55～59歳', '60～64歳', '65歳以上'])

Employment_man_df.reset_index(drop=True, inplace=True)
Employment_man_df['年度'] = Employment_man_df['年度'].str.replace('年', '').astype('int64')
multiply_specific_column(Employment_man_df, ages, 10000)

Employment_man_df

c:\Users\human\DataScience_FEA\venv\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,時間軸コード,年度,,15歳以上,15～64歳,15～24歳,15～19歳,20～24歳,25～34歳,25～29歳,...,35～44歳,35～39歳,40～44歳,45～54歳,45～49歳,50～54歳,55～64歳,55～59歳,60～64歳,65歳以上
0,2000000000,2000,NaN,10720000,**********************************************...,920000,180000,750000,2780000,1440000,...,2160000,1160000,1000000,2400000,1100000,1300000,1780000,1110000,670000,670000
1,2001000000,2001,NaN,10650000,**********************************************...,840000,160000,680000,2820000,1420000,...,2180000,1170000,1010000,2400000,1040000,1360000,1740000,1040000,690000,680000
2,2002000000,2002,NaN,10630000,**********************************************...,910000,160000,760000,2700000,1300000,...,2230000,1210000,1030000,2290000,1000000,1290000,1770000,1060000,710000,720000
3,2003000000,2003,NaN,10680000,**********************************************...,870000,160000,710000,2700000,1270000,...,2320000,1250000,1060000,2190000,970000,1210000,1850000,1110000,740000,740000
4,2004000000,2004,NaN,10670000,**********************************************...,800000,150000,650000,2670000,1220000,...,2390000,1300000,1090000,2100000,970000,1130000,1950000,1150000,800000,770000
5,2005000000,2005,NaN,10680000,**********************************************...,770000,160000,620000,2620000,1190000,...,2450000,1310000,1140000,2040000,980000,1060000,1990000,1210000,770000,800000
6,2006000000,2006,NaN,10710000,**********************************************...,750000,160000,580000,2570000,1170000,...,2550000,1370000,1170000,2000000,990000,1010000,2020000,1270000,750000,830000
7,2007000000,2007,NaN,11020000,**********************************************...,850000,150000,700000,2480000,1100000,...,2660000,1430000,1240000,2030000,1040000,990000,2100000,1250000,840000,900000
8,2008000000,2008,NaN,11000000,**********************************************...,810000,150000,660000,2410000,1090000,...,2740000,1450000,1290000,2040000,1070000,970000,2060000,1160000,900000,940000
9,2009000000,2009,NaN,10800000,9829829829829829829829829829829829829829829829...,720000,130000,590000,2290000,1050000,...,2760000,1450000,1310000,2060000,1090000,970000,1980000,1070000,910000,980000


有配偶率のデータフレーム準備

In [67]:
rows_to_read_3 = 1
MaritalStatus_df = pd.DataFrame()

for i in range(1, 22):
    #year.append(2020-(i-1))
    tmp_df = pd.DataFrame()
    tmp_df = pd.read_excel('FEI_PREF_230720160448.xlsx', 
                            skiprows=6, nrows=rows_to_read_3, sheet_name=str(i), header=None,  
                            names=['地域コード', '地域', '', '5～9歳人口', '', '15～19歳人口（男）', '', '15～19歳人口（女）', '', '20～24歳人口（男）', '', '20～24歳人口（女）', '', '30～34歳人口（男）', '', '30～34歳人口（女）', '', '有配偶人口（15歳以上）（男）', '', '有配偶人口（15歳以上）（女）', '', '有配偶人口（20～24歳）（男）', '', '有配偶人口（20～24歳）（女）', '', '有配偶人口（25～29歳）（男）', '', '有配偶人口（25～29歳）（女）', '', '有配偶人口（30～34歳）（男）', '', '有配偶人口（30～34歳）（女）', '', '離別人口（15歳以上）（男）', '', 'A1604002_離別人口（15歳以上）（女）', '', 'A1604011_離別人口（15～19歳）（男）', '', '離別人口（15～19歳）（女）', '', '離別人口（20～24歳）（男）', '', '離別人口（20～24歳）（女）', '', '離別人口（25～29歳）（男）', '', '離別人口（25～29歳）（女）', '', '離別人口（30～34歳）（男）', '', '離別人口（30～34歳）（女）', ''])
    
    MaritalStatus_df = pd.concat([MaritalStatus_df, tmp_df], ignore_index=True)

years = list(range(2020, 1999, -1))
MaritalStatus_df['年度'] = years

MaritalStatus_df = MaritalStatus_df.iloc[::-1]  # インデックスを逆順にする
MaritalStatus_df.reset_index(drop=True, inplace=True)

MaritalStatus_df

c:\Users\human\DataScience_FEA\venv\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\human\DataScience_FEA\venv\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\human\DataScience_FEA\venv\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\human\DataScience_FEA\venv\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\human\DataScience_FEA\venv\Lib\site-packages\openpyxl\s

,地域コード,地域,,5～9歳人口,.1,15～19歳人口（男）,.2,15～19歳人口（女）,.3,20～24歳人口（男）,...,.21,離別人口（25～29歳）（男）,.22,離別人口（25～29歳）（女）,.23,離別人口（30～34歳）（男）,.24,離別人口（30～34歳）（女）,.25,年度
0,11000,埼玉県,NaN,"338,965",NaN,"215,132",NaN,"203,531",NaN,"262,358",...,NaN,"2,544",NaN,"5,499",NaN,"5,081",NaN,"9,090",NaN,2000
1,11000,埼玉県,NaN,"340,000",NaN,"207,000",NaN,"194,000",NaN,"257,000",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2001
2,11000,埼玉県,NaN,"340,000",NaN,"200,000",NaN,"187,000",NaN,"250,000",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2002
3,11000,埼玉県,NaN,"340,000",NaN,"196,000",NaN,"182,000",NaN,"240,000",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003
4,11000,埼玉県,NaN,"336,000",NaN,"191,000",NaN,"176,000",NaN,"230,000",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2004
5,11000,埼玉県,NaN,"333,788",NaN,"189,256",NaN,"177,127",NaN,"224,751",...,NaN,"2,231",NaN,"4,649",NaN,"5,727",NaN,"11,678",NaN,2005
6,11000,埼玉県,NaN,"333,000",NaN,"185,000",NaN,"173,000",NaN,"224,000",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006
7,11000,埼玉県,NaN,"329,000",NaN,"183,000",NaN,"170,000",NaN,"221,000",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2007
8,11000,埼玉県,NaN,"326,000",NaN,"184,000",NaN,"170,000",NaN,"215,000",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008
9,11000,埼玉県,NaN,"320,000",NaN,"186,000",NaN,"172,000",NaN,"207,000",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009


重回帰分析の実行

In [94]:
# 就業率（説明変数）
target_age = ['20～24歳', '25～29歳', '30～34歳']

EmploymentRate_woman_df = Employment_woman_df[Employment_woman_df['年度'].isin(year_df['年度'])]
EmploymentRate_woman_df = EmploymentRate_woman_df[target_age]
EmploymentRate_woman_df.columns = ['20～24歳（女）', '25～29歳（女）', '30～34歳（女）']
EmploymentRate_woman_df.reset_index(drop=True, inplace=True)

EmploymentRate_man_df = Employment_man_df[Employment_man_df['年度'].isin(year_df['年度'])]
EmploymentRate_man_df = EmploymentRate_man_df[target_age]
EmploymentRate_man_df.columns = ['20～24歳（男）', '25～29歳（男）', '30～34歳（男）']
EmploymentRate_man_df.reset_index(drop=True, inplace=True)

# 有配偶率
#target_age_woman = [item + '人口（女）' for item in target_age]
#MaritalStatus_woman_df = MaritalStatus_df[target_age_woman]

# 出生数（目的変数）
BirthRate_df = Population_df[Population_df['年度'].isin(year_df['年度'])]
BirthRate_df = BirthRate_df['出生数']

In [95]:
x = pd.concat([year_df, EmploymentRate_woman_df, EmploymentRate_man_df], axis=1)
y = BirthRate_df
x

,年度,20～24歳（女）,25～29歳（女）,30～34歳（女）,20～24歳（男）,25～29歳（男）,30～34歳（男）
0,2009,590000,820000,780000,590000,1050000,1240000
1,2010,550000,820000,770000,530000,1030000,1200000
2,2011,520000,810000,740000,510000,1030000,1150000
3,2012,590000,810000,770000,590000,1010000,1160000
4,2013,600000,800000,780000,590000,980000,1140000
5,2014,600000,810000,780000,590000,990000,1120000
6,2015,590000,800000,780000,580000,980000,1110000
7,2016,610000,820000,800000,600000,1010000,1110000
8,2017,660000,810000,810000,680000,950000,1090000
9,2018,730000,830000,830000,740000,970000,1070000


In [97]:
# 重回帰分析を実施、独立変数間の分散拡大要因を確認後、分析結果を出力
model = sm.OLS(y, x)
num_cols = model.exog.shape[1] # 説明変数の列数
vifs = [vif(model.exog, i) for i in range(0, num_cols)]
print(pd.DataFrame(vifs, index=model.exog_names, columns=['VIF']))
result = model.fit()
print(result.summary())

                    VIF
年度         16392.127968
20～24歳（女）  14176.486938
25～29歳（女）  31771.450109
30～34歳（女）   9898.117393
20～24歳（男）   7028.813495
25～29歳（男）  18216.351787
30～34歳（男）   3282.255857
                                 OLS Regression Results                                
Dep. Variable:                    出生数   R-squared (uncentered):                   1.000
Model:                            OLS   Adj. R-squared (uncentered):              1.000
Method:                 Least Squares   F-statistic:                          8.101e+04
Date:                Thu, 20 Jul 2023   Prob (F-statistic):                    2.51e-12
Time:                        16:42:13   Log-Likelihood:                         -78.167
No. Observations:                  12   AIC:                                      170.3
Df Residuals:                       5   BIC:                                      173.7
Df Model:                           7                                                  
Covariance Type:

c:\Users\human\DataScience_FEA\venv\Lib\site-packages\scipy\stats\_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=12
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
